In [1]:
# import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
import h5py
import numpy as np

import h5py

import h5py
import numpy as np

# Load test data from test_raw.h5
with h5py.File("./data/test_raw.h5", "r") as f_test:
    X_test = f_test['data'][:]
    Y_test = f_test['labels'][:]
    S_test = f_test['sessions'][:]
    #H_test = f_test['hardwares'][:]  # Load the 'hardwares' dataset as h_test


# Now, X_combined, Y_combined, S_combined, and h_combined contain the merged data


# Now, X_combined, Y_combined, and S_combined contain the merged data


# Shuffle the data
indices = np.arange(X_test.shape[0])
np.random.shuffle(indices)

# Reorder the arrays according to the shuffled indices
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]
#h_test = H_test[indices]


# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []
h_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []
h_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    
    # Skip subjects with fewer than two unique sessions
    if len(np.unique(subject_sessions)) < 2:
        continue

    print(f"Subject {subject}, Number of unique sessions: {len(np.unique(subject_sessions))}")
    
    # Assign the minimum session to the evaluation set (x_test_e)
    min_session = np.min(subject_sessions)
    
    # Append data to the evaluation set (min session)
    x_test_e_list.extend(x_test[subject_indices][subject_sessions == min_session])
    y_test_e_list.extend(y_test[subject_indices][subject_sessions == min_session])
    s_test_e_list.extend(s_test[subject_indices][subject_sessions == min_session])
    #h_test_e_list.extend(h_test[subject_indices][subject_sessions == min_session])

    # Append remaining sessions to the validation set (x_test_v)
    x_test_v_list.extend(x_test[subject_indices][subject_sessions != min_session])
    y_test_v_list.extend(y_test[subject_indices][subject_sessions != min_session])
    s_test_v_list.extend(s_test[subject_indices][subject_sessions != min_session])
    #h_test_v_list.extend(h_test[subject_indices][subject_sessions != min_session])

# Shuffle and convert lists back to numpy arrays for x_test_e
indices_e = np.arange(len(x_test_e_list))
np.random.shuffle(indices_e)

x_test_e = np.array(x_test_e_list)[indices_e]
y_test_e = np.array(y_test_e_list)[indices_e]
s_test_e = np.array(s_test_e_list)[indices_e]
#h_test_e = np.array(h_test_v_list)[indices_e]

# Shuffle and convert lists back to numpy arrays for x_test_v
indices_v = np.arange(len(x_test_v_list))
np.random.shuffle(indices_v)

x_test_v = np.array(x_test_v_list)[indices_v]
y_test_v = np.array(y_test_v_list)[indices_v]
s_test_v = np.array(s_test_v_list)[indices_v]
#h_test_v = np.array(h_test_v_list)[indices_v]
# Optional: Save the new test evaluation and validation sets to npy files (if needed)


Subject 70, Number of unique sessions: 19
Subject 86, Number of unique sessions: 19
Subject 96, Number of unique sessions: 18
Subject 125, Number of unique sessions: 19
Subject 129, Number of unique sessions: 5
Subject 167, Number of unique sessions: 2
Subject 174, Number of unique sessions: 19
Subject 175, Number of unique sessions: 6
Subject 183, Number of unique sessions: 6
Subject 194, Number of unique sessions: 19
Subject 197, Number of unique sessions: 19
Subject 199, Number of unique sessions: 19
Subject 201, Number of unique sessions: 19
Subject 203, Number of unique sessions: 6
Subject 206, Number of unique sessions: 6
Subject 207, Number of unique sessions: 41
Subject 214, Number of unique sessions: 18
Subject 219, Number of unique sessions: 5
Subject 230, Number of unique sessions: 6
Subject 236, Number of unique sessions: 42
Subject 245, Number of unique sessions: 6
Subject 250, Number of unique sessions: 41
Subject 257, Number of unique sessions: 6
Subject 260, Number of u

In [2]:
with h5py.File("./data/neg_raw.h5", "r") as f:
    x_neg_r = f['data'][:]
    y_neg = f['labels'][:]
    s_neg = f['sessions'][:]

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from collections import defaultdict
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import cosine_similarity as cs


def EERf(resutls):
    resutls= np.array(resutls)
    genuine  = resutls[resutls[:, 1] == 1][:, 0]
    impostor = resutls[resutls[:, 1] == 0][:, 0]
    stats_a = get_eer_stats(genuine, impostor)
    return(stats_a.eer,stats_a.fmr100)

def calculate_and_print_averages(y_train, resutls3):
    u, counts = np.unique(y_train, return_counts=True)
    eer_values = []
    ii = 0

    for i in resutls3.keys():
        re = EERf(resutls3[i])
        eer = re[0]
        print(f"{i}: EER = {re[0]:.4f}, FMR100 = {re[1]:.4f}, Count = {counts[ii]}")
        eer_values.append(eer)
        ii += 1

    average_eer = np.mean(eer_values) * 100
    std_eer = np.std(eer_values) * 100
    
    print(f"Final Average EER: {average_eer:.4f}")
    print(f"Final EER Standard Deviation: {std_eer:.4f}")
    print(f"${average_eer:.2f} \\pm {std_eer:.2f}$")

import torch.nn.functional as F


def calculate_similarity_scores_two(enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance):
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    print(verification_embeddings.shape, enrollment_embeddings.shape)
    if distance == "cd":
        similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    elif distance == "ed":
        similarity_matrix = -1 * ed(verification_embeddings, enrollment_embeddings)

    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        predicted_scores = similarity_matrix[i]
        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]

        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            max_score = sum(sorted(predicted_scores[same_class_indices], reverse=True)[:10]) / 10
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls, i, cls])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls, i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two(enrollment_data, ye, verification_data, yv, e_network, distance):
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two(
        enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance
    )
    return similarity_results_by_class, similarity_results_by_class_dict

def compute_embedding_batch_two(x_test_batch, embedding_network=None):
    print(x_test_batch.shape)
    return x_test_batch

In [4]:
#HEREAA

# Generate a sequence of indices from 0 to the length of one of the arrays



#, s_test_b
resutls2,resutls3=assessment_model_data_two(x_test_e.reshape(x_test_e.shape[0], -1), y_test_e, x_test_v.reshape(x_test_v.shape[0], -1), y_test_v, None, distance = "ed")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

(5000, 46500)
(80643, 46500)
(80643, 46500) (5000, 46500)


/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


70: EER = 0.5233, FMR100 = 0.9922, Count = 100
86: EER = 0.4622, FMR100 = 0.9978, Count = 100
96: EER = 0.5695, FMR100 = 0.9994, Count = 100
125: EER = 0.3333, FMR100 = 0.9672, Count = 100
129: EER = 0.4500, FMR100 = 1.0000, Count = 100
167: EER = 0.7524, FMR100 = 1.0000, Count = 100
174: EER = 0.5596, FMR100 = 0.9967, Count = 100
175: EER = 0.9023, FMR100 = 1.0000, Count = 100
183: EER = 0.3760, FMR100 = 0.9200, Count = 100
194: EER = 0.5933, FMR100 = 0.9994, Count = 100
197: EER = 0.4717, FMR100 = 0.9883, Count = 100
199: EER = 0.4079, FMR100 = 0.9922, Count = 100
201: EER = 0.6261, FMR100 = 1.0000, Count = 100
203: EER = 0.5920, FMR100 = 0.9980, Count = 100
206: EER = 0.8500, FMR100 = 1.0000, Count = 100
207: EER = 0.4626, FMR100 = 0.9992, Count = 100
214: EER = 0.4318, FMR100 = 0.9595, Count = 100
219: EER = 0.6207, FMR100 = 1.0000, Count = 100
230: EER = 0.5348, FMR100 = 0.9340, Count = 100
236: EER = 0.5716, FMR100 = 0.9998, Count = 100
245: EER = 0.6360, FMR100 = 1.0000, Count =

In [5]:
#HEREAA

# Generate a sequence of indices from 0 to the length of one of the arrays



#, s_test_b
resutls2,resutls3=assessment_model_data_two(x_test_e.reshape(x_test_e.shape[0], -1), y_test_e, x_test_v.reshape(x_test_v.shape[0], -1), y_test_v, None, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

(5000, 46500)
(80643, 46500)
(80643, 46500) (5000, 46500)
70: EER = 0.4557, FMR100 = 0.9833, Count = 100
86: EER = 0.3545, FMR100 = 0.9206, Count = 100
96: EER = 0.4841, FMR100 = 0.9871, Count = 100
125: EER = 0.4667, FMR100 = 0.9939, Count = 100
129: EER = 0.3808, FMR100 = 0.9700, Count = 100
167: EER = 0.2800, FMR100 = 1.0000, Count = 100
174: EER = 0.3961, FMR100 = 0.9150, Count = 100
175: EER = 0.1880, FMR100 = 0.4960, Count = 100
183: EER = 0.3660, FMR100 = 0.9160, Count = 100
194: EER = 0.5478, FMR100 = 0.8356, Count = 100


/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


197: EER = 0.4661, FMR100 = 0.9794, Count = 100
199: EER = 0.3517, FMR100 = 0.9683, Count = 100
201: EER = 0.3996, FMR100 = 0.8461, Count = 100
203: EER = 0.2100, FMR100 = 0.4280, Count = 100
206: EER = 0.1646, FMR100 = 0.9120, Count = 100
207: EER = 0.4662, FMR100 = 0.9712, Count = 100
214: EER = 0.5217, FMR100 = 0.9881, Count = 100
219: EER = 0.5022, FMR100 = 1.0000, Count = 100
230: EER = 0.5305, FMR100 = 0.9920, Count = 100
236: EER = 0.4786, FMR100 = 0.9632, Count = 100
245: EER = 0.3180, FMR100 = 0.9320, Count = 100
250: EER = 0.4901, FMR100 = 0.9875, Count = 100
257: EER = 0.4920, FMR100 = 0.9840, Count = 100
260: EER = 0.4381, FMR100 = 0.9958, Count = 100
261: EER = 0.4736, FMR100 = 0.9856, Count = 100
262: EER = 0.3880, FMR100 = 1.0000, Count = 100
265: EER = 0.3513, FMR100 = 0.9436, Count = 100
268: EER = 0.3739, FMR100 = 0.9639, Count = 100
270: EER = 0.5780, FMR100 = 0.9800, Count = 100
281: EER = 0.3696, FMR100 = 0.9772, Count = 100
291: EER = 0.5344, FMR100 = 0.9739, Coun

In [6]:
# Flatten each array from 3D to 2D
x_test_e = x_test_e.reshape(x_test_e.shape[0], -1)
x_test_v= x_test_v.reshape(x_test_v.shape[0], -1)
x_test_n = x_neg_r.reshape(x_neg_r.shape[0], -1)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


def calculate_eer(y_true, y_scores):
    """
    Calculate the Equal Error Rate (EER) and FMR100 using pyeer.
    """
    # Separate genuine and impostor scores
    genuine_scores = y_scores[y_true == 1]
    impostor_scores = y_scores[y_true == 0]
    #print(y_true, y_scores)

    #print(genuine_scores, impostor_scores)
    print("genuine_scores: ", len(genuine_scores), sum(genuine_scores)/len(genuine_scores), "impostor_scores", len(impostor_scores), sum(impostor_scores)/len(impostor_scores))
    
    # Use pyeer to compute EER stats
    stats = get_eer_stats(genuine_scores, impostor_scores)
    
    return stats.eer

def classifiers_learn(enrollment_data, ye, verification_data, yv, e_network, negative_raw):
    """
    Learn classifiers per unique subject and evaluate using EER.
    """
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    
    # Compute embeddings
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    negative_embeddings = compute_embedding_batch_two(negative_raw, e_network)
    
    unique_classes = np.unique(y_enrollment)
    results = {}

    print("Unique classes: ", unique_classes)

    for cls in unique_classes:
        # Prepare training data
        positive_indices = np.where(y_enrollment == cls)[0]
        negative_indices = np.arange(len(negative_embeddings))
        
        train_embeddings = np.concatenate([enrollment_embeddings[positive_indices], negative_embeddings])
        train_labels = np.concatenate([np.ones(len(positive_indices)), np.zeros(len(negative_indices))])

        # Prepare testing data
        test_embeddings = verification_embeddings
        test_labels =(y_verification == cls).astype(int)


        # Initialize Incremental PCA
        #pca = IncrementalPCA(n_components=50)  # Retain 80% of variance or specify a fixed number of components
        

        #batch_size = 5000  # Define batch size
        #for i in range(0, train_embeddings.shape[0], batch_size):
            #batch = train_embeddings[i:i + batch_size]
            #pca.partial_fit(batch)  
        #train_embeddings_reduced = pca.transform(train_embeddings)
        #print("Reduced Training Shape:", train_embeddings_reduced.shape)
        #test_embeddings_reduced = pca.transform(test_embeddings)
        #print("Reduced Test Shape:", test_embeddings_reduced.shape)

        # Train Random Forest model
        #rf_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
        #rf_model.fit(train_embeddings, train_labels)
        #rf_probabilities = rf_model.predict_proba(test_embeddings)[:, 1]  # Use probabilities for EER calculation

        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=20))
            #('classifier', SVC(probability=True, random_state=42, class_weight='balanced'))
            #('classifier', KNeighborsClassifier(n_neighbors=20, weights='distance'))
        ])
        #svm_model = SVC(probability=True, random_state=42, class_weight='balanced')
        X, y = shuffle(train_embeddings, train_labels, random_state=42)
        pipeline.fit(X, y)
        probabilities = pipeline.predict_proba(test_embeddings)[:, 1]

        # Calculate EER
        rf_eer = calculate_eer(test_labels, probabilities)

        print(f"Class {cls}: RandomForest EER = {rf_eer:.4f}")

        results[cls] = {
            'RandomForest': {'EER': rf_eer}
        }

    return results

def calculate_mean_std(results):
    """
    Calculate the mean and standard deviation of EER values for each model in the results dictionary.
    
    Args:
        results (dict): A dictionary where keys are classes, and values are dictionaries containing model metrics.
                        Example: {cls: {'RandomForest': {'EER': value}}}
    
    Returns:
        dict: A dictionary containing mean and std for each model across all classes.
    """
    # Extract EER values for each model
    model_eer_values = {}
    for cls, metrics in results.items():
        for model, model_metrics in metrics.items():
            if model not in model_eer_values:
                model_eer_values[model] = []
            model_eer_values[model].append(model_metrics['EER'])
    
    # Calculate mean and std
    model_stats = {}
    for model, eer_values in model_eer_values.items():
        mean = np.mean(eer_values) * 100
        std = np.std(eer_values) * 100
        model_stats[model] = {'mean': mean, 'std': std}
        print(f"Final Average EER {model}: {mean:.4f}")
        print(f"Final EER Standard Deviation {model}: {std:.4f}")
        print(f"${mean:.2f} \\pm {std:.2f}$") 
        
   
    return model_stats


In [8]:
# Test your pipeline with synthetic data



resutls = classifiers_learn(
    x_test_e, y_test_e, x_test_v, y_test_v, None, x_test_n
)
# Print results
#print("EER:", EERf(resutls))

calculate_mean_std(resutls)

(5000, 46500)
(80643, 46500)
(84408, 46500)
Unique classes:  [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]
genuine_scores:  1800 0.003483333333333311 impostor_scores 78843 0.0016820770391792635
Class 70: RandomForest EER = 0.4595
genuine_scores:  1800 0.03631111111111108 impostor_scores 78843 0.0010192407696308205
Class 86: RandomForest EER = 0.2978
genuine_scores:  1700 0.001058823529411765 impostor_scores 78943 0.000680997681871727
Class 96: RandomForest EER = 0.4926
genuine_scores:  1800 0.0031722222222222095 impostor_scores 78843 0.0006595385766650122
Class 125: RandomForest EER = 0.4616
genuine_scores:  400 0.0028000000000000013 impostor_scores 80243 0.0013490273294868172
Class 129: RandomForest EER = 0.4497
genuine_scores:  100 0.006000000000000003 impostor_scores 80543 0.0014125374023814317
Class 167: RandomForest EER = 0.3

{'RandomForest': {'mean': 39.45661936233044, 'std': 9.285379443400636}}

In [9]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


def classifiers_learn(enrollment_data, ye, verification_data, yv, e_network, negative_raw):
    """
    Learn classifiers per unique subject and evaluate using EER.
    """
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    
    # Compute embeddings
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    negative_embeddings = compute_embedding_batch_two(negative_raw, e_network)
    
    unique_classes = np.unique(y_enrollment)
    results = {}

    print("Unique classes: ", unique_classes)

    for cls in unique_classes:
        # Prepare training data
        positive_indices = np.where(y_enrollment == cls)[0]
        negative_indices = np.arange(len(negative_embeddings))
        
        train_embeddings = np.concatenate([enrollment_embeddings[positive_indices], negative_embeddings])
        train_labels = np.concatenate([np.ones(len(positive_indices)), np.zeros(len(negative_indices))])

        # Prepare testing data
        test_embeddings = verification_embeddings
        test_labels =(y_verification == cls).astype(int)


        # Initialize Incremental PCA
        #pca = IncrementalPCA(n_components=50)  # Retain 80% of variance or specify a fixed number of components
        

        #batch_size = 5000  # Define batch size
        #for i in range(0, train_embeddings.shape[0], batch_size):
            #batch = train_embeddings[i:i + batch_size]
            #pca.partial_fit(batch)  
        #train_embeddings_reduced = pca.transform(train_embeddings)
        #print("Reduced Training Shape:", train_embeddings_reduced.shape)
        #test_embeddings_reduced = pca.transform(test_embeddings)
        #print("Reduced Test Shape:", test_embeddings_reduced.shape)

        # Train Random Forest model
        #rf_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
        #rf_model.fit(train_embeddings, train_labels)
        #rf_probabilities = rf_model.predict_proba(test_embeddings)[:, 1]  # Use probabilities for EER calculation

        
        # Replace RandomForest with Linear SVM + Kernel Approximation
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('kernel_approximation', RBFSampler(gamma=1, random_state=42, n_components=500)),  # Approximate RBF kernel
            ('classifier', SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, class_weight='balanced', random_state=42, n_jobs=20))
        ])
        
        # Shuffle and split the data
        X, y = shuffle(train_embeddings, train_labels, random_state=42)
        
        # Train the pipeline
        pipeline.fit(X, y)
        
        # Predict probabilities for the test set
        # Note: SGDClassifier doesn't provide probabilities directly, so we use decision_function
        decision_scores = pipeline.decision_function(test_embeddings)
        probabilities = 1 / (1 + np.exp(-decision_scores))  # Sigmoid function to map decision scores to probabilities
        
        # Calculate EER
        svm_eer = calculate_eer(test_labels, probabilities)
        
        print(f"Class {cls}: Linear SVM with Kernel Approximation EER = {svm_eer:.4f}")
        
        results[cls] = {
            'RandomForest': {'EER': svm_eer}
        }
        
    return results



In [10]:
# Test your pipeline with synthetic data



resutls = classifiers_learn(
    x_test_e, y_test_e, x_test_v, y_test_v, None, x_test_n
)
# Print results
#print("EER:", EERf(resutls))

calculate_mean_std(resutls)

(5000, 46500)
(80643, 46500)
(84408, 46500)
Unique classes:  [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]
genuine_scores:  1800 0.17438766836608888 impostor_scores 78843 0.17311816178706052
Class 70: Linear SVM with Kernel Approximation EER = 0.4963
genuine_scores:  1800 0.2810209985695079 impostor_scores 78843 0.2785233684722682
Class 86: Linear SVM with Kernel Approximation EER = 0.5022
genuine_scores:  1700 0.22085661710100507 impostor_scores 78943 0.21067184496068453
Class 96: Linear SVM with Kernel Approximation EER = 0.4953
genuine_scores:  1800 0.3020934116857288 impostor_scores 78843 0.29689912740728464
Class 125: Linear SVM with Kernel Approximation EER = 0.4986
genuine_scores:  400 0.1376187453122641 impostor_scores 80243 0.12323929084317273
Class 129: Linear SVM with Kernel Approximation EER = 0.4834
genuine_scores:  1

/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  1800 0.11533527789227521 impostor_scores 78843 0.117691557961794
Class 174: Linear SVM with Kernel Approximation EER = 0.5022


/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  500 0.2015529097867549 impostor_scores 80143 0.2045855678585946
Class 175: Linear SVM with Kernel Approximation EER = 0.5009


...

/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  800 0.1012322139000305 impostor_scores 79843 0.10765891886323935
Class 419: Linear SVM with Kernel Approximation EER = 0.5063


/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  700 0.20835722252289413 impostor_scores 79943 0.20515124144645655
Class 422: Linear SVM with Kernel Approximation EER = 0.4957
genuine_scores:  700 0.168187150574229 impostor_scores 79943 0.17056665944387786
Class 436: Linear SVM with Kernel Approximation EER = 0.5043
Final Average EER RandomForest: 50.0411
Final EER Standard Deviation RandomForest: 0.7420
$50.04 \pm 0.74$


{'RandomForest': {'mean': 50.041085021931295, 'std': 0.741986696812424}}